In [1]:
import nltk
import pandas as pd




In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [37]:
data=pd.read_csv("Ch05/05_04/Start/SMSSpamCollection.tsv",sep='\t',header=None)

In [38]:
data.columns=['Label',"Text"]

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5568 entries, 0 to 5567
Data columns (total 2 columns):
Label    5568 non-null object
Text     5568 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [5]:
import string

In [46]:
puncs=string.punctuation
def count_punct(text):
    count=sum([1 for char in text if char in puncs])
    return round(count/(len(text)-text.count(" ")),3)*100

In [77]:
data['body_len']=data['Text'].apply(lambda x:len(x)-x.count(" "))
data['punct_percent']=data['Text'].apply(lambda x:count_punct(x))


,Label,Text,body_len,punct_percent
0,ham,I've been searching for the right words to tha...,160,2.5
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,128,4.7
2,ham,"Nah I don't think he goes to usf, he lives aro...",49,4.1
3,ham,Even my brother is not like to speak with me. ...,62,3.2
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,28,7.1
5,ham,As per your request 'Melle Melle (Oru Minnamin...,135,4.4
6,spam,WINNER!! As a valued network customer you have...,132,4.5
7,spam,Had your mobile 11 months or more? U R entitle...,126,1.6
8,ham,I'm gonna be home soon and i don't want to tal...,89,6.7
9,spam,"SIX chances to win CASH! From 100 to 20,000 po...",111,7.2


In [52]:
from nltk.stem import WordNetLemmatizer
wn=WordNetLemmatizer()
stopwords=nltk.corpus.stopwords.words("english")
def clean_text(text):
    text="".join([i for i in text if i not in stopwords] )
    tokens=re.split("\W+",text)
    return [wn.lemmatize(word) for word in tokens if word not in stopwords]

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(analyzer=clean_text)
X=tfidf.fit_transform(data['Text'])

In [70]:
X_features=pd.concat([data['body_len'],data['punct_percent'],pd.DataFrame(X.toarray())],axis=1)


In [71]:
X_features.head(1)

,body_len,punct_percent,0,1,2,3,4,5,6,7,...,6900,6901,6902,6903,6904,6905,6906,6907,6908,6909
0,160,2.5,0.050526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.model_selection import GridSearchCV
model=RandomForestClassifier()

In [103]:
X_train, X_test, y_train, y_test = train_test_split(   X, data['Label'], test_size=0.33, random_state=42)
from sklearn.model_selection import GridSearchCV


In [108]:
gs=GridSearchCV(estimator=model,param_grid={
    "max_depth":[10,15,20,25,30],
    "n_estimators":[20,30,40,50,60]
})
gs.fit(X_train,y_train)

/home/elvin/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=20,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=50, n_jobs=-1,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='w

In [113]:
model=RandomForestClassifier(**gs.best_params_)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)


In [114]:
precision,recall,fscore,support=score(y_test,y_pred,pos_label='spam',average='binary')

In [115]:
print("precision {},recall {} , acc {} ".format(
    (precision),
    (precision),
    sum(y_pred==y_test)/len(y_pred)
))

precision 1.0,recall 1.0 , acc 0.9684439608269858 
